# Bash Functions

## Load Variables and Make Directories

In [ ]:
source bioinf_intro_config.sh
mkdir -p $STAR_OUT

## Trim and Map Reads

In [ ]:
TrimAndMap() {
    FASTQ=$1
    FASTQ_BASE="$(basename ${FASTQ} '_001.fastq.gz')"

#     FASTQ_BASE=${FASTQ##*/} # strip directory from file path
#     FASTQ_BASE="${FASTQ_BASE%_R1_001.fastq.gz}" # strip .fq.gz file extension
#     echo $FASTQ_BASE
    echo $FASTQ
    echo $FASTQ_BASE
    # exit 1


    # make a pipe for trimmed fastq
    CUR_PIPE=`mktemp --dry-run`_${FASTQ_BASE}_pipe.fq
    mkfifo $CUR_PIPE

    # Run fastq-mcf
    fastq-mcf \
        $ADAPTERS \
        $FASTQ \
        -o $CUR_PIPE \
        -q 20 -x 0.5 &
        
    STAR \
    --runMode alignReads \
    --twopassMode None \
    --genomeDir $GENOME_DIR \
    --outSAMtype None \
    --quantMode GeneCounts \
    --outFileNamePrefix ${STAR_OUT}/${FASTQ_BASE}_ \
    --alignIntronMax 5000 \
    --outSJfilterIntronMaxVsReadN 500 1000 2000 \
    --readFilesIn $CUR_PIPE        
        
    # Destroy pipe
    rm -f $CUR_PIPE
}
export -f TrimAndMap


## Call the function

In [ ]:
for FASTQ in $RAW_FASTQS/35_MA_P_S39_L00[1-2]_R1_001.fastq.gz
    do
        TrimAndMap $FASTQ
    done

In [ ]:
ls -ltr $STAR_OUT